In [1]:
import numpy as np
import pandas as pd

In [262]:
return_df = pd.read_csv('./data/kospi/preprocessed/df_return.csv',index_col=0)
jaccard_df = pd.read_csv('./data/similarity/jaccard_df.csv',index_col=0)
lv2_df = jaccard_df.copy().loc[return_df.index]

## Main table II 구성

Excess return, 3-factor alpha, 5-factor alpha 을 계산한 table II 를 만들고자 합니다.
유사도 기준으로 Q1~Q5까지 낮은순부터 높은 순대로 portfolio를 만들고 document가 filing된 시점부터 종목을 보유하여 3개월 동안의 수익률을 보겠습니다. 그룹을 sorting하는 데 있어서는 전체유사도 평균이 아닌 개별기업의 연속된 사업보고서 값을 선정하였습니다.

Quantile 1(Q1) refers to firms that have the least similarity between their document this year and
the one last year; hence this portfolio consists of the “big changers.” Quintile 5 (Q5) refers
to firms that have the most similarity in their documents across years, and hence this
portfolio represents the “little to no changers.

In [264]:
lv2_df[lv2_df.loc[:,'삼양홀딩스'] > 0]

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20110331,0.482558,NaN,0.615385,0.579749,0.630503,0.203113,0.705706,NaN,0.750000,0.576200,...,0.873362,NaN,NaN,0.462889,0.577154,NaN,0.633452,0.574394,0.755193,0.716630
20120330,0.349498,0.691877,0.500778,0.484272,0.604478,0.650970,0.404380,0.446479,0.527778,0.372517,...,NaN,NaN,0.692913,0.649770,0.555950,0.895184,0.592593,0.425806,0.579278,0.710660
20130401,0.541254,NaN,NaN,NaN,0.714286,0.597826,0.536538,0.645907,NaN,0.525890,...,0.725910,NaN,NaN,0.643997,0.551537,NaN,0.533141,0.571237,0.730924,NaN
20140331,0.623506,0.576659,0.528402,0.468793,0.643312,0.898990,0.697959,0.668571,0.963636,0.510526,...,0.763092,0.471028,0.881773,0.776367,0.604414,0.650526,0.604839,0.543253,0.754881,0.737799
20150331,0.688259,0.782927,0.616085,0.589710,0.665848,0.924460,0.677355,0.529118,0.202312,0.812367,...,0.640756,0.772939,0.130774,0.678119,0.518400,0.923913,0.818681,0.672179,0.678571,0.724490
20160330,0.632207,0.574423,0.579921,0.591931,0.571914,0.933852,0.791111,0.674319,0.317259,0.782341,...,0.726695,0.566667,0.724882,0.677509,0.487842,0.785276,0.794721,0.652893,0.599624,0.781513
20170331,0.643299,0.816705,0.541444,0.597531,0.629448,0.604712,0.692469,0.867362,0.585882,0.680357,...,0.729977,0.629002,0.743371,0.703704,0.556535,NaN,0.890244,0.864389,0.637708,0.701692
20180402,0.540034,0.715789,0.599490,NaN,NaN,0.739583,0.646035,0.815789,0.321534,0.839187,...,0.652427,0.507442,0.578341,0.780149,0.638095,NaN,0.828804,0.739049,NaN,0.753463
20190401,0.690635,0.532609,0.727488,NaN,0.510638,0.625402,0.678700,0.738854,NaN,0.811252,...,0.758364,0.564672,0.671296,0.776720,0.543662,0.870103,0.857909,0.810392,0.677365,0.821839


In [380]:
index_ls = []
value_ls = []
for i in lv2_df.columns:
    for j in lv2_df.index:
        if lv2_df.loc[j,i] > 0:
            value_ls.append(lv2_df.loc[j,i])
            index_ls.append(i + '_'+str(j))
            
unit_jaccard = pd.DataFrame([index_ls,value_ls] , index = {'title','jaccard'}).T
unit_jaccard = unit_jaccard.sort_values(by = 'jaccard', ascending =True).reset_index(drop = True)
unit_jaccard

,title,jaccard
0,GS_20150331,0.130774
1,CJ_20120330,0.133525
2,SK하이닉스_20150331,0.202312
3,한국테크놀로지그룹_20110331,0.203113
4,롯데칠성_20130401,0.212121
...,...,...
995,일양약품_20190401,0.985348
996,대한유화_20130329,0.985366
997,GS_20130329,0.994475
998,대한유화_20160329,0.995951


In [381]:
val_idx,val_col = np.where(np.isfinite(lv2_df)) #va_idx대신에 앞에서구한 sorting idx를 쓰자

In [383]:
def get_after_filing_return(windows=30) : 
    container = np.full(shape=(len(val_idx),windows),fill_value=np.nan)
    idx_ls = []
    for num,(idx,col) in enumerate(zip(val_idx,val_col)) : 
        tmp = return_df.iloc[idx:idx+windows,col].values
        if windows - len(tmp) > 0 : 
            tmp = np.array(tmp.tolist()+[np.nan]*(windows-len(tmp)))
        container[num] = tmp #한기업의 filing이후 window_size만큼의 수익률 저장
        
        tmp_idx = return_df.index[idx]
        tmp_col = return_df.columns[col]        
        idx_ls.append("{}_{}".format(tmp_col,tmp_idx))
    
    final_df = pd.DataFrame(data=container,index=idx_ls)    
    return final_df

In [377]:
def get_return_per_sim(window):

    df = get_after_filing_return(windows=window)

    sorted_values = [] #idx별 유사도값
    for i in df.index:
        for idx,j in enumerate(unit_jaccard.title): 
            if i == j:
                sorted_values.append(unit_jaccard.jaccard[idx])
    
    df = df.cumsum(1).iloc[:,-1].reset_index()
    df.columns = ['corp_rcp','cum_return']
    df['sim'] = sorted_values
    df = df.sort_values(by = 'sim',ascending=True).reset_index(drop=True)
    
    return df


In [407]:
get_after_filing_return(30).cumsum(1)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
LF_20110324,0.013986,0.001917,-0.006809,0.021360,0.031634,0.035024,0.051916,0.053577,0.035335,0.038713,...,0.155205,0.153701,0.122075,0.117409,0.148659,0.154719,0.154719,0.192370,0.170599,0.209175
S&T모티브_20110325,0.018272,0.021535,0.006901,-0.006300,0.030489,0.028876,0.022414,0.014284,0.007727,-0.020326,...,0.132625,0.161865,0.100785,0.064476,0.033079,0.042804,0.021937,-0.004293,-0.019444,-0.043376
LG디스플레이_20110328,0.001437,-0.020084,-0.033281,-0.010992,0.019531,0.023762,0.057470,0.057470,0.062905,0.076418,...,0.167242,0.140347,0.162960,0.143304,0.108217,0.135489,0.131697,0.110123,0.104935,0.141441
고려제강_20110329,0.015237,0.015237,0.017687,0.012722,0.007694,0.020270,0.024004,0.028938,0.037550,0.060730,...,0.073672,0.084306,0.042362,0.021667,0.027846,0.025382,0.035301,0.029167,0.005768,0.019648
유한양행_20110330,0.013336,-0.012985,-0.012985,-0.016368,0.000589,0.003927,0.000600,-0.009398,-0.022869,-0.043352,...,-0.049846,-0.056873,-0.042736,-0.039238,-0.053130,-0.039042,-0.063357,-0.088270,-0.117465,-0.121220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LF_20190401,0.014000,0.035696,0.026044,0.070878,0.087669,0.071155,0.059961,0.061848,0.058082,0.084547,...,-0.002627,-0.000615,0.023481,0.021521,-0.011878,-0.011878,-0.042366,-0.023498,-0.033786,-0.033786
후성_20190401,0.025469,0.030698,0.043702,0.041134,0.044995,0.047560,0.059068,0.056540,0.057807,0.067934,...,0.066318,0.073856,0.098794,0.115825,0.123002,0.112313,0.061893,0.060629,0.021389,0.016118
SK이노베이션_20190401,0.011142,0.016652,0.024871,0.022153,0.030328,0.033031,0.033031,0.024944,0.033097,0.030401,...,0.042824,0.018760,0.021499,0.005106,-0.011561,-0.017210,-0.056983,-0.048107,-0.065703,-0.074658
CJ제일제당_20190401,0.004644,0.009266,0.009266,0.006199,-0.001493,0.017111,0.018633,0.020153,0.004979,0.011142,...,-0.016572,-0.024446,-0.027621,-0.041952,-0.033875,-0.038682,-0.053175,-0.072783,-0.077783,-0.111284


In [384]:
get_after_filing_return(30)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
LF_20110324,0.013986,-0.012069,-0.008726,0.028169,0.010274,0.003390,0.016892,0.001661,-0.018242,0.003378,...,0.018377,-0.001504,-0.031627,-0.004666,0.031250,0.006061,0.000000,0.037651,-0.021771,0.038576
S&T모티브_20110325,0.018272,0.003263,-0.014634,-0.013201,0.036789,-0.001613,-0.006462,-0.008130,-0.006557,-0.028053,...,-0.001460,0.029240,-0.061080,-0.036309,-0.031397,0.009724,-0.020867,-0.026230,-0.015152,-0.023932
LG디스플레이_20110328,0.001437,-0.021521,-0.013196,0.022288,0.030523,0.004231,0.033708,0.000000,0.005435,0.013514,...,0.011125,-0.026895,0.022613,-0.019656,-0.035088,0.027273,-0.003793,-0.021574,-0.005188,0.036506
고려제강_20110329,0.015237,0.000000,0.002450,-0.004965,-0.005028,0.012576,0.003734,0.004934,0.008612,0.023180,...,0.010711,0.010634,-0.041944,-0.020695,0.006179,-0.002464,0.009919,-0.006134,-0.023399,0.013880
유한양행_20110330,0.013336,-0.026322,0.000000,-0.003383,0.016957,0.003338,-0.003327,-0.009998,-0.013471,-0.020482,...,0.010645,-0.007027,0.014138,0.003498,-0.013892,0.014088,-0.024315,-0.024912,-0.029195,-0.003756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LF_20190401,0.014000,0.021696,-0.009653,0.044834,0.016791,-0.016514,-0.011194,0.001887,-0.003766,0.026465,...,0.026860,0.002012,0.024096,-0.001961,-0.033399,0.000000,-0.030488,0.018868,-0.010288,0.000000
후성_20190401,0.025469,0.005229,0.013004,-0.002567,0.003861,0.002564,0.011509,-0.002528,0.001267,0.010127,...,0.027097,0.007538,0.024938,0.017032,0.007177,-0.010689,-0.050420,-0.001264,-0.039241,-0.005270
SK이노베이션_20190401,0.011142,0.005510,0.008219,-0.002717,0.008174,0.002703,0.000000,-0.008086,0.008152,-0.002695,...,0.002681,-0.024064,0.002740,-0.016393,-0.016667,-0.005650,-0.039773,0.008876,-0.017595,-0.008955
CJ제일제당_20190401,0.004644,0.004622,0.000000,-0.003067,-0.007692,0.018605,0.001522,0.001520,-0.015175,0.006163,...,0.006339,-0.007874,-0.003175,-0.014331,0.008078,-0.004808,-0.014493,-0.019608,-0.005000,-0.033501


In [405]:
df_15 = get_return_per_sim(15)
df_30 = get_return_per_sim(30)
df_60 = get_return_per_sim(60)
df_90 = get_return_per_sim(90)

### filing 이후 30일간 누적수익률 (Q1: big changer)

In [400]:
print('Q1:',df_30.cum_return[0:200].mean())
print('Q2:',df_30.cum_return[200:400].mean())
print('Q3:',df_30.cum_return[400:600].mean())
print('Q4:',df_30.cum_return[600:800].mean())
print('Q5:',df_30.cum_return[800:1000].mean())

Q1: -0.007112638793787532
Q2: 0.011922555566329951
Q3: 0.02367268110000796
Q4: 0.010049255476928493
Q5: 0.03504437778824723


### filing 이후 60일간 누적수익률

In [402]:
print('Q1:',df_60.cum_return[0:200].mean())
print('Q2:',df_60.cum_return[200:400].mean())
print('Q3:',df_60.cum_return[400:600].mean())
print('Q4:',df_60.cum_return[600:800].mean())
print('Q5:',df_60.cum_return[800:1000].mean())

Q1: -0.02822525421655661
Q2: -0.0063154620757229175
Q3: 0.017502228743970016
Q4: 0.0017982194556595227
Q5: 0.012813278507852348


In [404]:
print('Q1:',df_90.cum_return[0:200].mean())
print('Q2:',df_90.cum_return[200:400].mean())
print('Q3:',df_90.cum_return[400:600].mean())
print('Q4:',df_90.cum_return[600:800].mean())
print('Q5:',df_90.cum_return[800:1000].mean())

Q1: -0.025266805131746194
Q2: -0.0224639581177133
Q3: 0.01479445173084587
Q4: 0.003013645084734996
Q5: 0.017129948102023194


In [406]:
print('Q1:',df_15.cum_return[0:200].mean())
print('Q2:',df_15.cum_return[200:400].mean())
print('Q3:',df_15.cum_return[400:600].mean())
print('Q4:',df_15.cum_return[600:800].mean())
print('Q5:',df_15.cum_return[800:1000].mean())

Q1: 0.005337455863004154
Q2: 0.016767255972322495
Q3: 0.024546313490574515
Q4: 0.0162563906782961
Q5: 0.0327314058567715
